In [1]:
%matplotlib notebook

In [2]:
%load_ext autoreload

%autoreload 2

In [2]:
pip install -U scikit-multiflow

Requirement already up-to-date: scikit-multiflow in c:\users\dell\desktop\fyp\scikit-multiflow dev\scikit-multiflow\src (0.6.dev0)
Note: you may need to restart the kernel to use updated packages.


# Preprocessing

In [3]:
import pandas as pd
import numpy as np

In [4]:
dataset = pd.read_csv('pulsar_data_train.csv')

In [5]:
from sklearn.impute import SimpleImputer
#imputer = SimpleImputer(missing_values = "NaN", strategy = "mean")

In [7]:
imputer = SimpleImputer(strategy="most_frequent")
print(imputer.fit_transform(dataset))

[[ 1.21156250e+02  4.83729711e+01  3.75484665e-01 ...  7.44987415e+00
   6.51592977e+01  0.00000000e+00]
 [ 7.69687500e+01  3.61755566e+01  7.12897860e-01 ...  9.41465226e+00
   1.02722975e+02  0.00000000e+00]
 [ 1.30585938e+02  5.32295335e+01  1.33408289e-01 ...  8.50836378e+00
   7.40313242e+01  0.00000000e+00]
 ...
 [ 1.16031250e+02  4.32138464e+01  6.63455691e-01 ...  1.70552145e+01
   3.12204325e+02  0.00000000e+00]
 [ 1.35664062e+02  4.99337494e+01 -8.99403060e-02 ...  7.39839490e+00
   6.23340175e+01  0.00000000e+00]
 [ 1.20726562e+02  5.04722564e+01  3.46178079e-01 ...  1.76622219e+01
   3.29548016e+02  0.00000000e+00]]


C:\Users\dell\Anaconda3\lib\site-packages\sklearn\impute\_base.py:356: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  row_mask = np.logical_not(row_mask).astype(np.bool)
C:\Users\dell\Anaconda3\lib\site-packages\sklearn\impute\_base.py:356: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  row_mask = np.logical_not(row_mask).astype(np.bool)
C:\Users\del

# Training

In [9]:
#Alrady tested. fast,not incrimently learn, not solve imbalance problem
from skmultiflow.data import FileStream

# 1. Create a stream

stream = FileStream('pulsar_data_train.csv')
#(./name)
print("Features:")
print(stream.feature_names)
print("Number of Targets: " + str(stream.n_targets)+"\nName: "+str(stream.target_names))
print("Target class values: " + str(stream.target_values))


Features:
[' Mean of the integrated profile', ' Standard deviation of the integrated profile', ' Excess kurtosis of the integrated profile', ' Skewness of the integrated profile', ' Mean of the DM-SNR curve', ' Standard deviation of the DM-SNR curve', ' Excess kurtosis of the DM-SNR curve', ' Skewness of the DM-SNR curve']
Number of Targets: 1
Name: ['target_class']
Target class values: [0, 1]


### ExtremelyFastDecisionTreeClassifier

In [62]:
#fast(not much as ht),incremantly learn, not slove imbalance problem 
from skmultiflow.trees.extremely_fast_decision_tree import ExtremelyFastDecisionTreeClassifier 
import numpy as np
#from skmultiflow.trees.split_criterion import GiniSplitCriterion
from skmultiflow.data import FileStream
#from skmultiflow.trees import ExtremelyFastDecisionTreeClassifier
from skmultiflow.evaluation import EvaluatePrequential
# 1. Create a stream
import datetime
start_time = datetime.datetime.now()
stream = FileStream('pulsar_data_train.csv')

efdth = ExtremelyFastDecisionTreeClassifier(split_criterion='hellinger')


print("info: ")
print(efdth.get_info)
# Setup variables to control loop and track performance
n_samples = 0
correct_cnt = 0
max_samples = 20000
y_true_all = []
y_pred_all = []
# Train the estimator with the samples provided by the data stream
while n_samples < max_samples and stream.has_more_samples():
    X, y = stream.next_sample()
    y_pred = efdth.predict(X)
    y_true_all.append(y[0])
    y_pred_all.append(y_pred[0])
    if y[0] == y_pred[0]:
        correct_cnt += 1
    efdth.partial_fit(X, y)
    n_samples += 1
end_time = datetime.datetime.now()
time_difference = end_time - start_time
print('{} samples analyzed.'.format(n_samples))
print('{} Time difference'.format(time_difference))
print('Extremely Fast Decision Tree accuracy: {}'.format(correct_cnt / n_samples))
print("info: ")
print(efdth.get_info)


info: 
<bound method BaseSKMObject.get_info of ExtremelyFastDecisionTreeClassifier(binary_split=False, grace_period=200,
                                    leaf_prediction='nba',
                                    max_byte_size=33554432,
                                    memory_estimate_period=1000000,
                                    min_samples_reevaluate=20, nb_threshold=0,
                                    nominal_attributes=None,
                                    split_confidence=1e-07,
                                    split_criterion='hellinger',
                                    stop_mem_management=False,
                                    tie_threshold=0.05)>
9273 samples analyzed.
0:01:02.937233 Time difference
Extremely Fast Decision Tree accuracy: 0.97239296883425
info: 
<bound method BaseSKMObject.get_info of ExtremelyFastDecisionTreeClassifier(binary_split=False, grace_period=200,
                                    leaf_prediction='nba',
                 

### Testing Results

In [65]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(y_true_all,y_pred_all).ravel()
recall = recall_score(y_true_all, y_pred_all)
precision = precision_score(y_true_all, y_pred_all)
accuracy = accuracy_score(y_true_all, y_pred_all)
f1_score = f1_score(y_true_all, y_pred_all)
specificity = tn/(tn + fp)
G_mean = np.sqrt((precision * specificity))

print('Accuracy: {0} \nRecall: {1} \nPrecision: {2} \nF1 Score: {3} \nSpecificity(TNR): {4} \nG-Mean: {5}'
      .format(accuracy,recall,precision,f1_score,specificity,G_mean))

Accuracy: 0.97239296883425 
Recall: 0.808235294117647 
Precision: 0.8807692307692307 
F1 Score: 0.8429447852760736 
Specificity(TNR): 0.9889588032767422 
G-Mean: 0.9332976397829983


# ExtremelyFastDecisionTreeClassifier

In [53]:
start_time = datetime.datetime.now()
stream = FileStream('pulsar_data_train.csv')

efdtgh = ExtremelyFastDecisionTreeClassifier(split_criterion='gaussian_hellinger')


print("info: ")
print(efdtgh.get_info)
# Setup variables to control loop and track performance
n_samples = 0
correct_cnt = 0
max_samples = 20000
y_true_all = []
y_pred_all = []
# Train the estimator with the samples provided by the data stream
while n_samples < max_samples and stream.has_more_samples():
    X, y = stream.next_sample()
    y_pred = efdtgh.predict(X)
    y_true_all.append(y[0])
    y_pred_all.append(y_pred[0])
    if y[0] == y_pred[0]:
        correct_cnt += 1
    efdtgh.partial_fit(X, y)
    n_samples += 1
end_time = datetime.datetime.now()
time_difference = end_time - start_time
print('{} samples analyzed.'.format(n_samples))
print('{} Time difference'.format(time_difference))
print('Extremely Fast Decision Tree accuracy: {}'.format(correct_cnt / n_samples))
print("info: ")
print(efdtgh.get_info)

info: 
<bound method BaseSKMObject.get_info of ExtremelyFastDecisionTreeClassifier(binary_split=False, grace_period=200,
                                    leaf_prediction='nba',
                                    max_byte_size=33554432,
                                    memory_estimate_period=1000000,
                                    min_samples_reevaluate=20, nb_threshold=0,
                                    nominal_attributes=None,
                                    split_confidence=1e-07,
                                    split_criterion='gaussian_hellinger',
                                    stop_mem_management=False,
                                    tie_threshold=0.05)>
9273 samples analyzed.
0:02:57.974059 Time difference
Extremely Fast Decision Tree accuracy: 0.9674323304216542
info: 
<bound method BaseSKMObject.get_info of ExtremelyFastDecisionTreeClassifier(binary_split=False, grace_period=200,
                                    leaf_prediction='nba',
      

### Testing Results

In [61]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(y_true_all,y_pred_all).ravel()
recall = recall_score(y_true_all, y_pred_all)
precision = precision_score(y_true_all, y_pred_all)
accuracy = accuracy_score(y_true_all, y_pred_all)
f1_score = f1_score(y_true_all, y_pred_all)
specificity = tn/(tn + fp)
G_mean = np.sqrt((precision * specificity))

print('Accuracy: {0} \nRecall: {1} \nPrecision: {2} \nF1 Score: {3} \nSpecificity(TNR): {4} \nG-Mean: {5}'
      .format(accuracy,recall,precision,f1_score,specificity,G_mean))


Accuracy: 0.9674323304216542 
Recall: 0.7505882352941177 
Precision: 0.8763736263736264 
F1 Score: 0.808618504435995 
Specificity(TNR): 0.9893149709129764 
G-Mean: 0.9311334752250742


In [68]:
start_time = datetime.datetime.now()
stream = FileStream('pulsar_data_train.csv')

efdtgini = ExtremelyFastDecisionTreeClassifier(split_criterion='gini')


print("info: ")
print(efdtgini.get_info)
# Setup variables to control loop and track performance
n_samples = 0
correct_cnt = 0
max_samples = 20000
y_true_all = []
y_pred_all = []
# Train the estimator with the samples provided by the data stream
while n_samples < max_samples and stream.has_more_samples():
    X, y = stream.next_sample()
    y_pred = efdtgini.predict(X)
    y_true_all.append(y[0])
    y_pred_all.append(y_pred[0])
    if y[0] == y_pred[0]:
        correct_cnt += 1
    efdtgini.partial_fit(X, y)
    n_samples += 1
end_time = datetime.datetime.now()
time_difference = end_time - start_time
print('{} samples analyzed.'.format(n_samples))
print('{} Time difference'.format(time_difference))
print('Extremely Fast Decision Tree accuracy: {}'.format(correct_cnt / n_samples))
print("info: ")
print(efdtgini.get_info)

info: 
<bound method BaseSKMObject.get_info of ExtremelyFastDecisionTreeClassifier(binary_split=False, grace_period=200,
                                    leaf_prediction='nba',
                                    max_byte_size=33554432,
                                    memory_estimate_period=1000000,
                                    min_samples_reevaluate=20, nb_threshold=0,
                                    nominal_attributes=None,
                                    split_confidence=1e-07,
                                    split_criterion='gini',
                                    stop_mem_management=False,
                                    tie_threshold=0.05)>
9273 samples analyzed.
0:00:47.445000 Time difference
Extremely Fast Decision Tree accuracy: 0.9718537690067939
info: 
<bound method BaseSKMObject.get_info of ExtremelyFastDecisionTreeClassifier(binary_split=False, grace_period=200,
                                    leaf_prediction='nba',
                    

### Testing Results

In [69]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(y_true_all,y_pred_all).ravel()
recall = recall_score(y_true_all, y_pred_all)
precision = precision_score(y_true_all, y_pred_all)
accuracy = accuracy_score(y_true_all, y_pred_all)
f1_score = f1_score(y_true_all, y_pred_all)
specificity = tn/(tn + fp)
G_mean = np.sqrt((precision * specificity))

print('Accuracy: {0} \nRecall: {1} \nPrecision: {2} \nF1 Score: {3} \nSpecificity(TNR): {4} \nG-Mean: {5}'
      .format(accuracy,recall,precision,f1_score,specificity,G_mean))

Accuracy: 0.9718537690067939 
Recall: 0.8152941176470588 
Precision: 0.8695106649937264 
F1 Score: 0.8415300546448088 
Specificity(TNR): 0.9876528552772171 
G-Mean: 0.9267009717136622


In [11]:
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
from skmultiflow.trees.extremely_fast_decision_trees import ExtremelyFastDecisionTrees 
ExtremelyFastDecisionTrees()

Main    : before creating thread x

Main    : before creating thread y

Main    : before running thread x

Main    : before running thread y

Main    : wait for the thread to finish
18197 samples analyzed.
Extremely Fast Decision Tree accuracy: 0.973072484475463
info: 
<bound method BaseSKMObject.get_info of ExtremelyFastDecisionTreeClassifier(binary_split=False, grace_period=200,
                                    leaf_prediction='nba',
                                    max_byte_size=33554432,
                                    memory_estimate_period=1000000,
                                    min_samples_reevaluate=20, nb_threshold=0,
                                    nominal_attributes=None,
                                    split_confidence=1e-07,
                                    split_criterion='hellinger',
                                    stop_mem_management=False,
                                    tie_threshold=0.05)>
18546 samples analyzed.
Extremely Fast Decis

### HoeffdingTreeClassifier

In [18]:
from skmultiflow.trees.hoeffding_tree import HoeffdingTreeClassifier
from skmultiflow.evaluation import EvaluatePrequential

# 2. Instantiate the HoeffdingTreeClassifier
ht = HoeffdingTreeClassifier()
# 3. Setup the evaluator
evaluator = EvaluatePrequential(show_plot=True,
                                pretrain_size=200,
                                max_samples=20000)
# 4. Run evaluation
evaluator.evaluate(stream=stream, model=ht)


<IPython.core.display.Javascript object>

Prequential Evaluation
Evaluating 1 target(s).
Pre-training on 200 sample(s).
Evaluating...
Processed samples: 200
Mean performance:


TypeError: 'NoneType' object is not subscriptable